# Emerging topics final code - Full (new code cleaning)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)

import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time

from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression as lm

import matplotlib.collections as mcol
from matplotlib.legend_handler import HandlerLineCollection, HandlerTuple
from matplotlib.lines import Line2D

import seaborn as sns
import os

from gensim.models.coherencemodel import CoherenceModel

import statsmodels.api as sm

## Data ingestion

In [ ]:
# full corpus
df = pd.read_pickle("/project/biocomplexity/sdad/projects_data/ncses/prd/Tech-Report/FR_meta_and_final_tokens_21SEPT14.pkl")

In [ ]:
df.head()

In [ ]:
# input needed for LDA, NMF (all from Scikit-Learn) is one string per document (not a list of strings)

text = []
docs = df["final_tokens"]

for abstract in docs:
    text.append(" ".join(abstract))

In [ ]:
def createLDAvars(docs):

    # Create the variables needed for LDA from df[final_frqwds_removed]: dictionary (id2word), corpus
    
    # Create Dictionary
    id2word = gensim.corpora.Dictionary(docs)

    #Filter words to only those found in at least a set number of documents (min_appearances)
    id2word.filter_extremes(no_below=20, no_above=0.6)
    
    # filter out stop words - "use" already filtered out by previous line
    id2word.filter_tokens(bad_ids=[id2word.token2id['research'], id2word.token2id['study'], \
                               id2word.token2id['project']])

    return id2word

In [ ]:
id2word = createLDAvars(docs)

## Functions needed for all models

In [ ]:
# Function to format topics as a "list of list of strings".
# Needed for topic coherence function in Gensim

# function modified from https://nlpforhackers.io/topic-modeling/

def list_topics(topic_term_dist, vectorizer, top_n=10):

    #input. top_n: how many words to list per topic.  If -1, then list all words.
       
    topic_words = []
    
    for idx, topic in enumerate(topic_term_dist):  # loop through each row of H.  idx = row index.  topic = actual row
            
        if top_n == -1:   
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[::-1]])
        else:
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n - 1:-1]])
        
    return topic_words

In [ ]:
def str_topics(topic_term_mat, vectorizer, top_n=10):

    #input. top_n: how many words to list per topic.  If -1, then list all words.
       
    topic_words = []
    
    for idx, topic in enumerate(topic_term_mat):  # loop through each row of H.  idx = row index.  topic = actual row
            
        if top_n == -1:   
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[::-1]])
        else:
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n - 1:-1]])
    
    str_wds = []
    
    for wds in topic_words:
        str_wds.append(", ".join(wds))    
    
    return str_wds 

## NMF
- 50 topics, random state = ?

In [ ]:
stop_wds = ['research', 'study', 'project']  # use will be eliminated by max_df

In [ ]:
# construct term document matrix
tfidf_vectorizer = TfidfVectorizer(max_df=0.6, min_df=20, lowercase=False, stop_words=stop_wds)
tf_idf = tfidf_vectorizer.fit_transform(text)

In [ ]:
# first time topic model run
num_topics = 50

nmf_model = NMF(n_components = num_topics, random_state = 1)
doc_topic = nmf_model.fit_transform(tf_idf)
topic_term = nmf_model.components_

with open("/project/biocomplexity/sdad/projects_data/ncses/prd/nmf_full_50.pkl","wb") as f:
    pickle.dump((doc_topic, topic_term), f)

In [ ]:
# read in fit topic model
#num_topics = 50

#with open("/project/biocomplexity/sdad/projects_data/ncses/prd/nmf_full_50.pkl", "rb") as f:
#    res = pickle.load(f)
    
#doc_topic = res[0]
#topic_term = res[1]

In [ ]:
topics = list_topics(topic_term, tfidf_vectorizer, top_n = 10)

In [ ]:
# calculate coherence
#cm = CoherenceModel(topics = topics,
#                    #corpus = corpus,
#                    dictionary = id2word,
#                    texts = docs, 
#                    coherence = 'c_v', 
#                    processes = 8)

## Emerging topics

In [ ]:
# Extract Year from PROJECT_START_DATE

def getYear(a):   
    a = str(a)
    if a.find("/"):
        splitdate = a.split("/")
        if len(splitdate) == 3:
            a = splitdate[2]
        else:
            a = splitdate[0]
    year = str(a)
    return year

df['START_YEAR'] = df['PROJECT_START_DATE'].apply(getYear)

In [ ]:
# add start year to doc topic matrix
topic_frame = pd.DataFrame(doc_topic, columns=["Topic"+" "+str(i) for i in range(num_topics)])
topic_frame["START_YEAR"] = df["START_YEAR"]

#### compute count of projects with weight > 0 for each topic

In [ ]:
topic_proj_count = topic_frame.copy()

In [ ]:
# count number of docs with weight > 0
topic_count_bool = (topic_proj_count.iloc[:,0:num_topics] > 0)
topic_count_bool["START_YEAR"] = topic_proj_count["START_YEAR"]

In [ ]:
# count number of docs with weight > 0 BY YEAR
topic_counts = topic_count_bool.groupby("START_YEAR").sum().reset_index()
topic_counts["START_YEAR"] = topic_counts["START_YEAR"].astype(int)

In [ ]:
# get docs between 2010 and 2019
topic_counts_filt = topic_counts[topic_counts["START_YEAR"] > 2009] 
topic_counts_filt = topic_counts_filt[topic_counts_filt["START_YEAR"] < 2020]

In [ ]:
n_by_topic = topic_counts_filt.sum()[1:]

#### calculate mean topic weight by year and standard errors on means

In [ ]:
topic_frame_se = topic_frame.groupby("START_YEAR").sem().reset_index()
topic_frame_se = topic_frame_se.sort_values(by = "START_YEAR")
topic_frame_se["START_YEAR"] = topic_frame_se["START_YEAR"].astype(int)

topic_wts_se_2010_2019 = topic_frame_se[topic_frame_se["START_YEAR"] > 2009] 
#topic_wts_se_2010_2019 = topic_wts_se_2010_2019[topic_wts_se_2010_2019["START_YEAR"] < 2020]
topic_wts_se_2010_2019 = topic_wts_se_2010_2019[topic_wts_se_2010_2019["START_YEAR"] < 2019]

In [ ]:
topic_frame = topic_frame.groupby("START_YEAR").mean().reset_index()
topic_frame = topic_frame.sort_values(by = "START_YEAR")
topic_frame["START_YEAR"] = topic_frame["START_YEAR"].astype(int)

In [ ]:
# filter topic_frame for years 2010 - 2019

topic_wts_2010_2019 = topic_frame[topic_frame["START_YEAR"] > 2009] 
#topic_wts_2010_2019 = topic_wts_2010_2019[topic_wts_2010_2019["START_YEAR"] < 2020] 
topic_wts_2010_2019 = topic_wts_2010_2019[topic_wts_2010_2019["START_YEAR"] < 2019] 

In [ ]:
# fit OLS and get slope, SE, p-value
topic_slopes = []
topic_slopes_se = []
topic_slopes_pval = []

lm_x = topic_wts_2010_2019["START_YEAR"].values.reshape(-1,1)
lm_x = sm.add_constant(lm_x)

for i in range(1,num_topics+1):

    linear_fit = sm.OLS(topic_wts_2010_2019.iloc[:,i].values.reshape(-1,1),lm_x).fit()
    
    topic_slopes.append(linear_fit.params[1])
    topic_slopes_se.append(linear_fit.bse[1])
    topic_slopes_pval.append(linear_fit.pvalues[1])

In [ ]:
topic_ols_res = pd.DataFrame(
    {"Slope": topic_slopes, 
     "SE": topic_slopes_se,
     "p-value": topic_slopes_pval
    })

In [ ]:
reg_topics = pd.DataFrame()

topic_label_num = range(1, num_topics + 1)
topic_label = ["FR" + str(x) for x in topic_label_num]
reg_topics["Topic Label"] = topic_label

reg_topics["Topic Words"] = str_topics(topic_term, tfidf_vectorizer, top_n = 5)

In [ ]:
regression_res = pd.concat([reg_topics, topic_ols_res], axis = 1)

In [ ]:
regression_res.to_csv("full_50_topics_2018_df.csv", index=False)

## Create plot - top 10 hot and cold topics

In [ ]:
# legend
leg = str_topics(topic_term, tfidf_vectorizer, top_n = 5)

In [ ]:
legend_df = reg_topics.set_index('Topic Words')
legend_df = legend_df.loc[leg]
leg_topic_label = legend_df['Topic Label'].tolist()

In [ ]:
leg_label = [i + ": " + j for i, j in zip(leg_topic_label, leg)]

In [ ]:
# collect slopes to sort
topic_results = {}

for i in range(1,num_topics+1):
    linear_fit = lm().fit(topic_wts_2010_2019["START_YEAR"].values.reshape(-1,1), topic_wts_2010_2019.iloc[:,i].values.reshape(-1,1))
    topic_results[i] = [linear_fit.coef_[0][0], reg_topics.loc[i-1, "Topic Label"]]

In [ ]:
def sort_dict(x):
    return sorted(x.items(), key=lambda l: l[1], reverse=True)

In [ ]:
sort_slopes = sort_dict(topic_results)
top_slopes = [x[0] for x in sort_slopes[:10]]
bottom_slopes = [x[0] for x in sort_slopes[-10:]]
topnbot_slopes = top_slopes + bottom_slopes

### Plot hottest and coldest topics on separate plots

In [ ]:
# Make Graphs

color_list = plt.rcParams['axes.prop_cycle'].by_key()['color']

line_return = []
fig = plt.figure()
fig.suptitle('Top 10 Topics with Increasing Weights', fontsize=16)
ax = fig.add_subplot(111)
plt.grid(True, color = "whitesmoke")
line = [[(0, 0)]]
i = 0
for n in top_slopes:
    zorder = 10
    color = color_list[i] # "#D3D3D3"   
    if i == 0:
        zorder = 10 #20
        color = color_list[i];
    linear_fit = lm().fit(topic_wts_2010_2019["START_YEAR"].values.reshape(-1,1), topic_wts_2010_2019.iloc[:,n].values.reshape(-1,1))
    #ax.plot(topic_wts_2010_2019["START_YEAR"], (topic_wts_2010_2019["START_YEAR"]*linear_fit.coef_[0][0])+linear_fit.intercept_,linestyle = 'dashed', color = color_list[i])
    ax.plot(topic_wts_2010_2019["START_YEAR"], topic_wts_2010_2019.iloc[:,n], '-o', color = color, zorder = zorder)
    ax.errorbar(topic_wts_2010_2019["START_YEAR"], topic_wts_2010_2019.iloc[:,n], 
            yerr = np.array(topic_wts_se_2010_2019.iloc[:,n]), fmt = "o", color = color, zorder = zorder)
    #line_return.append(mcol.LineCollection(2 * line, linestyles=['solid','dashed'], colors=[color_list[i],color_list[i]]))
    line_return.append(mcol.LineCollection(line, linestyles=['solid'], colors=[color]))
    i+=1
    #leg.append("Topic %d"%(n+1))

plt.xticks(np.arange(topic_wts_2010_2019["START_YEAR"].min(), topic_wts_2010_2019["START_YEAR"].max()+1, 1.0))
plt.xlabel('Year', fontsize=14)
plt.ylabel('Mean Topic Weight', fontsize=14)
plt.ylim(bottom = 0, top = 0.0035)
#ax.yaxis.set_label_coords(-0.14,0.5)

# set up the proxy artist
lc = mcol.LineCollection(2 * line, linestyles=['solid','dashed'], colors=['blue','blue'])
lc2 = mcol.LineCollection(2 * line, linestyles=['solid','dashed'], colors=['orange','orange'])
# create the legend
#plt.legend(line_return, [leg[x-1] for x in top_slopes], handler_map={type(line_return[0]): HandlerDashedLines()},
#          handlelength=2, handleheight=2,bbox_to_anchor=(1.05, 0.7, 0.3, 0.2), loc='upper left')

plt.legend([leg_label[x-1] for x in top_slopes], bbox_to_anchor=(0.35, -0.35, 0.3, 0.2), 
           loc='upper center', fontsize = 'large', frameon = False)
plt.savefig("full_increasing_50_topics_2018.png", dpi = 800, bbox_inches = "tight")
plt.show()


#Make Graphs
#leg = []

#bottom_slopes.reverse()  # so the plot legend has the coldest listed first

line_return = []
fig = plt.figure()
fig.suptitle('Top 10 Topics with Decreasing Weights', fontsize=16)
ax = fig.add_subplot(111)
plt.grid(True, color = "whitesmoke")
i = 0
for n in bottom_slopes:
    zorder = 10
    color = color_list[i]  #"#D3D3D3"  
    if i == 4:
        zorder = 10 #20
        color = color_list[i]
    linear_fit = lm().fit(topic_wts_2010_2019["START_YEAR"].values.reshape(-1,1), topic_wts_2010_2019.iloc[:,n].values.reshape(-1,1))
    #ax.plot(topic_wts_2010_2019["START_YEAR"], (topic_wts_2010_2019["START_YEAR"]*linear_fit.coef_[0][0])+linear_fit.intercept_,linestyle = 'dashed', color = color_list[i+5])
    ax.plot(topic_wts_2010_2019["START_YEAR"], topic_wts_2010_2019.iloc[:,n], '-o', color=color, zorder = zorder)
    ax.errorbar(topic_wts_2010_2019["START_YEAR"], topic_wts_2010_2019.iloc[:,n], 
            yerr = np.array(topic_wts_se_2010_2019.iloc[:,n]), fmt = "o", color=color, zorder = zorder)
    #line_return.append(mcol.LineCollection(2 * line, linestyles=['solid','dashed'], colors=[color_list[i+5],color_list[i+5]]))
    line_return.append(mcol.LineCollection(line, linestyles=['solid'], colors=[color]))
    i+=1
    #leg.append("Topic %d"%(n+1))

plt.xticks(np.arange(topic_wts_2010_2019["START_YEAR"].min(), topic_wts_2010_2019["START_YEAR"].max()+1, 1.0))
plt.xlabel('Year', fontsize=14)
plt.ylabel('Mean Topic Weight', fontsize=14)
plt.ylim(bottom = 0, top = 0.0035)
#ax.yaxis.set_label_coords(-0.14,0.5)


# set up the proxy artist
lc = mcol.LineCollection(2 * line, linestyles=['solid','dashed'], colors=['blue','blue'])
lc2 = mcol.LineCollection(2 * line, linestyles=['solid','dashed'], colors=['orange','orange'])

# create the legend
#plt.legend(line_return, [leg[x-1] for x in bottom_slopes], handler_map={type(line_return[0]): HandlerDashedLines()},
#          handlelength=2, handleheight=2,bbox_to_anchor=(1.05, 0.7, 0.3, 0.2), loc='upper left')

plt.legend([leg_label[x-1] for x in bottom_slopes], bbox_to_anchor=(0.35, -0.35, 0.3, 0.2), 
           loc='upper center', fontsize = 'large', frameon = False)
plt.savefig("full_decreasing_50_topics_2018.png", dpi = 800, bbox_inches = "tight")
plt.show()
